<a href="https://colab.research.google.com/github/helloworld2j/TRINITcodecraftersML3/blob/main/trinit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!unzip "/content/drive/MyDrive/dataset.zip" -d "content/"

In [3]:
import pandas as  pd

data = pd.read_csv("/content/content/train.csv")

In [4]:
data.shape

(8734, 3)

In [5]:

data.sample(10)

,filename,captions,image
2255,rsicd_images/denseresidential_369.jpg,['The neighborhood is a residential neighborho...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
7249,rsicd_images/stadium_235.jpg,['several buildings and green trees are around...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
6682,rsicd_images/sparseresidential_179.jpg,['many green trees and meadows are around a bu...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
3725,rsicd_images/mediumresidential_251.jpg,['Many green buildings and trees are located i...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
7857,rsicd_images/viaduct_321.jpg,['Some green trees and many bare lands are nea...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
5708,rsicd_images/railwaystation_209.jpg,['Many crowded buildings are located on both s...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
6761,rsicd_images/sparseresidential_25.jpg,['a building with a swimming pool and a car is...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
6479,rsicd_images/school_211.jpg,['many buildings and green trees are around a ...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
8186,rsicd_images/00241.jpg,['each square of field character grid owns two...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1849,rsicd_images/commercial_255.jpg,['Many buildings and some green trees are loca...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...


In [6]:
import csv
import ast
import os
from PIL import Image
from io import BytesIO
csv.field_size_limit(100000000)

131072

In [7]:
csv_file_path = '/content/content/train.csv'

In [8]:
output_folder = 'output_images'

In [9]:
os.makedirs(output_folder, exist_ok=True)

In [10]:
os.makedirs("output_images/rsicd_images", exist_ok=True)

In [11]:
def bytes_to_image(byte_stream):
    return Image.open(BytesIO(byte_stream))
h=0

In [12]:
with open(csv_file_path, 'r', newline='') as file:
    csv_reader = csv.DictReader(file)
    for row_num, row in enumerate(csv_reader, start=1):
        filename = row.get('filename')
        byte_stream_str = row.get('image')

        try:
            byte_stream_dict = ast.literal_eval(byte_stream_str)
            byte_stream_bytes = byte_stream_dict.get('bytes')

            if filename and byte_stream_bytes:
                image = bytes_to_image(byte_stream_bytes)
                image_path = os.path.join(output_folder, filename)
                image.save(image_path, 'JPEG')
                # print(f"Image saved: {image_path}")

        except (SyntaxError, ValueError) as e:
            # print(f"Error processing row {row_num}: {e}")
            continue

In [13]:
del csv_reader

In [14]:
del byte_stream_str

In [15]:

import cv2
import numpy as np
from rich.progress import track

def read_images_as_rgb(image_path: str) -> np.array:
  """
  Reads images from a list of paths as RGB format and converts them to a numpy array.

  Args:
    image_paths: A list of paths to images.

  Returns:
    A numpy array containing the images as RGB format.
  """

  image = cv2.imread(image_path, cv2.IMREAD_COLOR)
  image=cv2.resize(image,(224,224))
  # Convert the image from BGR to RGB format
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  return image

def convert_str_list_str_to_str(str_list_str: str) -> str:
    """Converts a string representation of a list of strings to a single string.

    Args:
        str_list_str (str): A string representing a list of strings, in the format "['str1', 'str2', ...]".

    Returns:
        str: A single string containing all the elements of the list, separated by spaces.
    """
    # Remove the outer quotes and brackets
    str_list_str = str_list_str[1:-1]

    # Split the string on commas and remove any leading/trailing whitespace
    str_list = [s.strip() for s in str_list_str.split(',')]

    # Join the strings in the list into a single string
    return ' '.join(str_list)



path :  str = "/content/output_images/"
features , captions  = [] , []


for _, i in track(data.iterrows(), description = "loading..."):
  # print(i["filename"])
  features.append(read_images_as_rgb(path + i["filename"]))
  captions.append(convert_str_list_str_to_str(i["captions"]))

features = np.array(features)


Output()

In [16]:
features.shape

(8734, 224, 224, 3)

In [17]:
captions[0]

"'Many aircraft are parked next to a long building in an airport.'\n 'Many planes are parked next to a long building at an airport.'\n 'Many planes are parked next to a long building in an airport.'\n 'many planes are parked next to a long building at an airport.'\n 'many planes are parked next to a long building in an airport .'"

In [18]:
i["captions"]

"['two baseball fields are located in the huage area of grassland next to road .'\n 'two baseball fields are in one side of a road .'\n 'two baseball grounds are next to a road and a lake.'\n 'two baseball fields are next to a road and a pool .'\n 'two baseball fields are next to a road and a lake .']"

In [19]:
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

In [20]:
# load vgg16 model
vgg_model = VGG16()
# restructure the model
vgg_model = Model(inputs=vgg_model.inputs, outputs=vgg_model.layers[-2].output)
# summarize
print(vgg_model.summary())

553467096/553467096 [==============================] - 5s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [ ]:
extracted_features: list = []


for image in track(features):
  image = image.reshape((1, 224, 224, 3))
  extracted_features.append(vgg_model.predict(image, verbose=0))

extracted_features = np.array(extracted_features)

Output()

In [ ]:
del features
del data

In [ ]:
extracted_features.shape

In [ ]:
extracted_features = extracted_features.reshape((extracted_features.shape[0], 4096))

In [ ]:
len(captions)

In [ ]:
from typing import List

def clean(data: List) -> List:
  for i in range(len(data)):
      # take one caption at a time
      caption = data[i]
      # preprocessing steps
      # convert to lowercase
      caption = caption.lower()
      # delete digits, special chars, etc.,
      caption = caption.replace('[^A-Za-z]', '')
      # delete additional spaces
      caption = caption.replace('\s+', ' ')
      # add start and end tags to the caption
      caption = 'startseq ' + " ".join([word for word in caption.split() if len(word)>1]) + ' endseq'
      data[i] = caption
  return data

captions = clean(data=captions)


In [ ]:
captions[0]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
vocab_size

In [ ]:
max_length = max(len(caption.split()) for caption in captions)
max_length

In [ ]:
def data_generator(features, tokenizer, max_length, vocab_size, batch_size):
    # loop over images
    X1, X2, y = list(), list(), list()
    n = 0
    while 1:
      for caption in captions:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([caption])[0]
        # split the sequence into X, y pairs
        for i in range(1, len(seq)):
            # split into input and output pairs
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

            # store the sequences
            X1.append(features[0])
            X2.append(in_seq)
            y.append(out_seq)

      if n == batch_size:
        X1, X2, y = np.array(X1), np.array(X2), np.array(y)
        yield [X1, X2], y
        X1, X2, y = list(), list(), list()
        n = 0

In [ ]:
# model.fit(x =[extracted_features, X2], y=y, epochs=20, batch_size=10, validation_split=0.1)

# train the model
epochs = 70
batch_size = 32
steps = len(captions) // batch_size

def data_generator(features, tokenizer, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0
    while True:
        for caption in captions:
            seq = tokenizer.texts_to_sequences([caption])[0]
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                X1.append(features[0])
                X2.append(in_seq)
                y.append(out_seq)
                n += 1
                if n == batch_size:
                    yield [np.array(X1), np.array(X2)], np.array(y)
                    X1, X2, y = list(), list(), list()
                    n = 0


# encoder model
inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# sequence feature layers
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

# decoder model
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

for i in track(range(epochs)):
    generator = data_generator(extracted_features, tokenizer, max_length, vocab_size, batch_size)
    steps_per_epoch = len(captions) // batch_size
    for _ in range(steps_per_epoch):
        X, y = next(generator)
        model.train_on_batch(X, y,)

    #print(f"Epoch {i+1}/{epochs} completed.")


In [ ]:
# Save entire model
model.save('caption_model.h5')

# Or save just the weights
#model.save_weights('my_model_weights.h5')

In [ ]:
extracted_features[0].shape

In [ ]:
def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [ ]:
def predict_caption(model, image, tokenizer, max_length):
    # add start tag for generation process
    in_text = 'startseq'
    # iterate over the max length of sequence
    for i in range(max_length):
        # encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad the sequence
        sequence = pad_sequences([sequence], max_length)
        # predict next word
        yhat = model.predict([image, sequence], verbose=0)
        # get index with high probability
        yhat = np.argmax(yhat)
        # convert index to word
        word = idx_to_word(yhat, tokenizer)
        # stop if word not found
        if word is None:
            break
        # append word as input for generating next word
        in_text += " " + word
        # stop if we reach end tag
        if word == 'endseq':
            break


    return in_text

In [ ]:
print(predict_caption(model = model , image= extracted_features[90].reshape(1, 4096), tokenizer= tokenizer, max_length= max_length))

In [ ]:
path_ = "/content/output_images/rsicd_images/00059.jpg"

In [ ]:
tokenizer_path = 'tokenizer.pickle'
with open(tokenizer_path, 'wb') as handle:
    tokenizer_model = tokenizer.to_json()
    handle.write(tokenizer_model.encode('utf-8'))

In [ ]:
test_img=read_images_as_rgb(path_).reshape(1,224,224,3)

In [ ]:
test_feature=vgg_model.predict(test_img)

In [ ]:
test_feature.shape

In [ ]:
print(predict_caption(model = model , image= test_feature, tokenizer= tokenizer, max_length= 119))

In [ ]:
from google.colab import files
files.download('/content/caption_model.h5')